In [16]:
import geopandas as gpd
import pandas as pd
from progressbar import progressbar as pbar

In [17]:
z = pd.read_parquet("/opt/data/clay-california-worldcover-rgbnir-vvvh-chips-embeddings-v1-combined.parquet")

In [18]:
m = gpd.read_parquet("/opt/data/california-worldcover-chips/california-worldcover-chips-osm-multilabels_mod-prec7pc_emb-v0-2-20240426.parquet")

In [19]:
z.head()

,chips,embeddings
0,s3://clay-california-worldcover-rgbnir-vvvh-ch...,"[-0.092387736, 0.06289999, 0.14823247, 0.05708..."
1,s3://clay-california-worldcover-rgbnir-vvvh-ch...,"[-0.09912827, 0.055867158, 0.16590388, 0.04337..."
2,s3://clay-california-worldcover-rgbnir-vvvh-ch...,"[-0.11831675, 0.06658777, 0.13290095, 0.036696..."
3,s3://clay-california-worldcover-rgbnir-vvvh-ch...,"[-0.11574778, 0.06557509, 0.10578355, 0.031810..."
4,s3://clay-california-worldcover-rgbnir-vvvh-ch...,"[-0.11810879, 0.06218237, 0.16002795, 0.039691..."


In [20]:
m.head()

,onehot_count,onehot_area,onehot_length,string_labels,col,row,geometry,split
chip_id,,,,,,,,
193c3415fc237,"[24, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 53, ...","[58033.967200952815, 58033.967200952815, 0.0, ...","[4913.205774298301, 4913.205774298301, 0.0, 0....","[amenity=*, amenity=parking, building=*, build...",78080,96000,"POLYGON ((-118.47200 33.97867, -118.47200 34.0...",train
1639b12583426,"[148, 148, 4171, 1245, 1891, 52, 938, 13, 32, ...","[160073.78667008967, 160073.78667008967, 80191...","[20966.344659970102, 20966.344659970102, 24881...","[amenity=*, amenity=parking, building=*, build...",78080,95744,"POLYGON ((-118.47200 34.00000, -118.47200 34.0...",train
253904d963d6a,"[65, 65, 200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29...","[101980.95118259436, 101980.95118259436, 71246...","[10597.989172272657, 10597.989172272657, 16664...","[amenity=*, amenity=parking, building=*, build...",77824,95744,"POLYGON ((-118.49333 34.00000, -118.49333 34.0...",train
00530dca88d4a,"[9, 9, 4897, 688, 4026, 0, 172, 10, 1, 0, 0, 0...","[17637.908172952746, 17637.908172952746, 10930...","[1925.4953839123748, 1925.4953839123748, 32065...","[amenity=*, amenity=parking, building=*, build...",77824,95488,"POLYGON ((-118.49333 34.02133, -118.49333 34.0...",train
0e554bbdbe661,"[80, 80, 4358, 1262, 2386, 59, 597, 27, 27, 0,...","[131950.60085068244, 131950.60085068244, 98472...","[13536.03447682216, 13536.03447682216, 283601....","[amenity=*, amenity=parking, building=*, build...",78080,95488,"POLYGON ((-118.47200 34.02133, -118.47200 34.0...",train


In [21]:
z.shape

(111698, 2)

In [8]:
z.iloc[0]['embeddings'].shape

(768,)

In [11]:
chip_aws_path = z.iloc[0]['chips']
chip_aws_path

's3://clay-california-worldcover-rgbnir-vvvh-chips/chips/worldcover_california_chip_102400_99072.tif'

In [16]:
col, row = [int(i) for i in chip_aws_path.split("/")[-1].split(".")[0].split("_")[-2:]]

In [17]:
col, row

(102400, 99072)

In [23]:
chipid_map = {}
not_found = []
too_many = []
for chip_aws_path in pbar(z.chips.values):
    col, row = [int(i) for i in chip_aws_path.split("/")[-1].split(".")[0].split("_")[-2:]]
    c = m[ (m['col']==col) & (m['row']==row) ]
    if len(c)==1:
        chipid_map[(col,row)] = c.index[0]
    elif len(c)==0:
embedding        not_found.append([col,row])
    elif len(c)==0:
        too_many.append([col,row])


100% (111698 of 111698) |################| Elapsed Time: 0:00:53 Time:  0:00:530004


In [24]:
len(not_found), len(too_many), len(chipid_map)

(28207, 0, 83491)

In [26]:
not_found[0]

[102656, 97536]

In [27]:
m.shape

(83609, 8)

In [29]:
m['split'].value_counts()

split
train    58690
test     13078
val      11841
Name: count, dtype: int64

In [38]:
embeddings_folder = "/opt/data/california-worldcover-chips/embeddings_v1.0"

In [44]:
import pickle
for _,i in pbar(z.iterrows(), max_value=len(z)):
    fname = i.chips.split("/")[-1].split(".")[0]+".pkl"
    with open(f"{embeddings_folder}/{fname}", "wb") as f:
        pickle.dump(i.embeddings, f)
    

100% (111698 of 111698) |################| Elapsed Time: 0:00:14 Time:  0:00:140001


In [1]:
from omegaconf import OmegaConf
import hydra

In [4]:
conf = OmegaConf.load("../../configs/osmencoder-x.yaml")
conf.dataloader

{'_target_': 'earthtext.datamodules.chipmultilabel.ChipMultilabelModule', 'metadata_file': '/opt/data/california-worldcover-chips/california-worldcover-chips-osm-multilabels_mod-prec7pc_emb-v0-2-20240426.parquet', 'embeddings_folder': '/opt/data/california-worldcover-chips/embeddings_v0.2-20240426', 'chips_folder': '/opt/data/clay-california-worldcover-rgbnir-vvvh-chips/chips', 'get_osm_strlabels': True, 'get_osm_ohearea': True, 'get_osm_ohecount': True, 'get_osm_ohelength': True, 'embeddings_normalization': 'mean_stdev_norm', 'multilabel_threshold_osm_ohecount': 1, 'normalize_input': True, 'batch_size': 32}

In [46]:
embeddings_folder = "/opt/data/california-worldcover-chips/embeddings_v1.0"

conf.dataloader.embeddings_folder = embeddings_folder
conf.dataloader.get_chip_id = True
conf.dataloader['embeddings_folder'], conf.dataloader.chips_folder, conf.dataloader.get_chip_id
conf.dataloader.embeddings_normalization = None

In [47]:
data = hydra.utils.instantiate(conf.dataloader)

2024-05-09 12:54:32.849 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:93 - checking chip files for train split
100% (58690 of 58690) |##################| Elapsed Time: 0:00:03 Time:  0:00:030000
2024-05-09 12:54:36.206 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:97 - read train split with 58598 chip files (out of 58690)
2024-05-09 12:54:36.207 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:102 - max cache size is -1
2024-05-09 12:54:36.644 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:93 - checking chip files for val split
100% (11841 of 11841) |##################| Elapsed Time: 0:00:00 Time:  0:00:000000
2024-05-09 12:54:37.318 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:97 - read val split with 11822 chip files (out of 11841)
2024-05-09 12:54:37.318 | INFO     | earthtext.datamodules.components.chipmultilabel:__init__:102 - max cache size is -1
2024-05-09 12:54:3

In [48]:
data.train_dataset.get_chip_id

True

In [49]:
len(data.train_dataset) + len(data.val_dataset) + len(data.test_dataset)

83491

In [50]:
data.disable_chip_loading()

In [83]:
traindl = data.train_dataloader()

In [75]:
for batch in pbar(traindl):
    pass

100% (1832 of 1832) |####################| Elapsed Time: 0:00:15 Time:  0:00:150001


In [77]:
testdl = data.test_dataloader()
for batch in pbar(testdl):
    pass

100% (409 of 409) |######################| Elapsed Time: 0:00:03 Time:  0:00:030000


In [51]:
valdl = data.val_dataloader()
for batch in pbar(valdl):
    pass

100% (370 of 370) |######################| Elapsed Time: 0:00:02 Time:  0:00:020000


In [52]:
batch.keys()

dict_keys(['multilabel', 'chip_id', 'embedding', 'osm_strlabels', 'osm_ohearea', 'osm_ohecount', 'osm_ohelength'])

In [53]:
ci = np.random.randint(len(batch['chip_id'])
chip_id = batch['chip_id'][ci]
col = m.loc[chip_id]['col']
row = m.loc[chip_id]['row']
fname = f"s3://clay-california-worldcover-rgbnir-vvvh-chips/chips/worldcover_california_chip_{col}_{row}.tif"
np.allclose(z[z.chips==fname].iloc[0].embeddings, batch['embedding'][0])
col, row

(37632, 11776)

In [55]:
fname = f"s3://clay-california-worldcover-rgbnir-vvvh-chips/chips/worldcover_california_chip_{col}_{row}.tif"
fname

's3://clay-california-worldcover-rgbnir-vvvh-chips/chips/worldcover_california_chip_37632_11776.tif'

In [56]:
import numpy as np
np.allclose(z[z.chips==fname].iloc[0].embeddings, batch['embedding'][0])

True

In [57]:
 batch['embedding'][0][:10], z[z.chips==fname].iloc[0].embeddings[:10]

(tensor([-0.0998,  0.0810, -0.0187,  0.1351,  0.0835,  0.3186,  0.0655,  0.1875,
         -0.2679, -0.0590]),
 array([-0.09982982,  0.08101191, -0.01867879,  0.13510755,  0.08345962,
         0.31858194,  0.06550831,  0.18751638, -0.26793486, -0.05899944],
       dtype=float32))

In [ ]:
 batch['embedding'][0][:10]

In [39]:
z[z.chips==fname]

,chips,embeddings
85000,s3://clay-california-worldcover-rgbnir-vvvh-ch...,"[-0.099829815, 0.08101191, -0.01867879, 0.1351..."
